In [1]:
import os
import glob
import pandas as pd

In [2]:
# Define working directory
os.chdir("C:/Users/samya/VS Code Projects/Python/Python Projects/Spotify_hit_predictor/backend")

In [3]:
ds00 = pd.read_csv('Original_data/dataset-of-00s.csv')
ds10 = pd.read_csv('Original_data/dataset-of-10s.csv')
ds60 = pd.read_csv('Original_data/dataset-of-60s.csv')
ds70 = pd.read_csv('Original_data/dataset-of-70s.csv')
ds80 = pd.read_csv('Original_data/dataset-of-80s.csv')
ds90 = pd.read_csv('Original_data/dataset-of-90s.csv')

merge = [ds00, ds10, ds60, ds70, ds80, ds90]
df = pd.concat(merge)
df.reset_index(drop=True, inplace=True)

In [4]:
df2 = df.drop(['track', 'artist', 'uri'], axis=1)

In [5]:
# Sin tempo & speechiness
data = df2
X = data[['danceability', 'energy','key', 'loudness','mode', 'acousticness', 'instrumentalness', 'liveness','valence', 'duration_ms', 'time_signature','chorus_hit', 'sections']]  
y = data.target

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

## Data Preprocessing

In [7]:
#Scale features (X) using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler(feature_range=(0,1)).fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test) 

X_train_scaled

array([[0.78947368, 0.52283395, 0.81818182, ..., 0.8       , 0.19905999,
        0.06508876],
       [0.54048583, 0.83094117, 0.81818182, ..., 0.8       , 0.21633373,
        0.04142012],
       [0.5       , 0.66588373, 0.81818182, ..., 0.8       , 0.20480589,
        0.03550296],
       ...,
       [0.57692308, 0.36777999, 0.81818182, ..., 0.8       , 0.15844718,
        0.09467456],
       [0.41902834, 0.16070793, 0.63636364, ..., 0.8       , 0.200772  ,
        0.04733728],
       [0.29554656, 0.24873856, 0.81818182, ..., 0.6       , 0.12021911,
        0.06508876]])

One-hot encode the labels

In [8]:
#One-hot encode output labels (y)
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

y_train_categorical

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

## Creating and defining our Deep Learning Model Architecture

In [9]:
#Create a sequential model
from tensorflow.keras.models import Sequential
model = Sequential() 

from tensorflow.keras.layers import Dense
number_inputs = 13  

#Create hidden layers
model.add(Dense(units=14,activation='relu', input_dim=number_inputs))
model.add(Dense(units=120,activation='relu'))
model.add(Dense(units=80,activation='relu'))

#Create output layer
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax')) 

Number of Hidden Nodes 
https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw#:~:text=The%20number%20of%20hidden%20neurons,size%20of%20the%20input%20layer.

In [10]:
#Model Summary
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 14)                196       
                                                                 
 dense_1 (Dense)             (None, 120)               1800      
                                                                 
 dense_2 (Dense)             (None, 80)                9680      
                                                                 
 dense_3 (Dense)             (None, 2)                 162       
                                                                 
Total params: 11,838
Trainable params: 11,838
Non-trainable params: 0
_________________________________________________________________


In [11]:
#Compile the Model
import tensorflow as tf
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## Training the Model

In [12]:
#Training the Model
history = model.fit(X_train_scaled, y_train_categorical, epochs=500, batch_size=2000, shuffle=True, verbose=2)

Epoch 1/500
16/16 - 0s - loss: 0.6626 - accuracy: 0.6302 - 445ms/epoch - 28ms/step
Epoch 2/500
16/16 - 0s - loss: 0.6087 - accuracy: 0.6705 - 47ms/epoch - 3ms/step
Epoch 3/500
16/16 - 0s - loss: 0.5646 - accuracy: 0.6986 - 52ms/epoch - 3ms/step
Epoch 4/500
16/16 - 0s - loss: 0.5424 - accuracy: 0.7181 - 45ms/epoch - 3ms/step
Epoch 5/500
16/16 - 0s - loss: 0.5337 - accuracy: 0.7244 - 43ms/epoch - 3ms/step
Epoch 6/500
16/16 - 0s - loss: 0.5282 - accuracy: 0.7325 - 42ms/epoch - 3ms/step
Epoch 7/500
16/16 - 0s - loss: 0.5241 - accuracy: 0.7358 - 40ms/epoch - 3ms/step
Epoch 8/500
16/16 - 0s - loss: 0.5206 - accuracy: 0.7385 - 41ms/epoch - 3ms/step
Epoch 9/500
16/16 - 0s - loss: 0.5172 - accuracy: 0.7400 - 42ms/epoch - 3ms/step
Epoch 10/500
16/16 - 0s - loss: 0.5153 - accuracy: 0.7420 - 47ms/epoch - 3ms/step
Epoch 11/500
16/16 - 0s - loss: 0.5131 - accuracy: 0.7424 - 41ms/epoch - 3ms/step
Epoch 12/500
16/16 - 0s - loss: 0.5114 - accuracy: 0.7439 - 35ms/epoch - 2ms/step
Epoch 13/500
16/16 - 0s

Epoch 101/500
16/16 - 0s - loss: 0.4837 - accuracy: 0.7613 - 58ms/epoch - 4ms/step
Epoch 102/500
16/16 - 0s - loss: 0.4825 - accuracy: 0.7633 - 52ms/epoch - 3ms/step
Epoch 103/500
16/16 - 0s - loss: 0.4823 - accuracy: 0.7629 - 55ms/epoch - 3ms/step
Epoch 104/500
16/16 - 0s - loss: 0.4818 - accuracy: 0.7634 - 52ms/epoch - 3ms/step
Epoch 105/500
16/16 - 0s - loss: 0.4817 - accuracy: 0.7632 - 52ms/epoch - 3ms/step
Epoch 106/500
16/16 - 0s - loss: 0.4821 - accuracy: 0.7629 - 55ms/epoch - 3ms/step
Epoch 107/500
16/16 - 0s - loss: 0.4800 - accuracy: 0.7642 - 47ms/epoch - 3ms/step
Epoch 108/500
16/16 - 0s - loss: 0.4795 - accuracy: 0.7646 - 46ms/epoch - 3ms/step
Epoch 109/500
16/16 - 0s - loss: 0.4810 - accuracy: 0.7639 - 49ms/epoch - 3ms/step
Epoch 110/500
16/16 - 0s - loss: 0.4813 - accuracy: 0.7626 - 48ms/epoch - 3ms/step
Epoch 111/500
16/16 - 0s - loss: 0.4809 - accuracy: 0.7629 - 51ms/epoch - 3ms/step
Epoch 112/500
16/16 - 0s - loss: 0.4802 - accuracy: 0.7635 - 50ms/epoch - 3ms/step
Epoc

Epoch 200/500
16/16 - 0s - loss: 0.4628 - accuracy: 0.7761 - 51ms/epoch - 3ms/step
Epoch 201/500
16/16 - 0s - loss: 0.4624 - accuracy: 0.7757 - 42ms/epoch - 3ms/step
Epoch 202/500
16/16 - 0s - loss: 0.4623 - accuracy: 0.7757 - 60ms/epoch - 4ms/step
Epoch 203/500
16/16 - 0s - loss: 0.4622 - accuracy: 0.7762 - 38ms/epoch - 2ms/step
Epoch 204/500
16/16 - 0s - loss: 0.4624 - accuracy: 0.7760 - 49ms/epoch - 3ms/step
Epoch 205/500
16/16 - 0s - loss: 0.4618 - accuracy: 0.7771 - 50ms/epoch - 3ms/step
Epoch 206/500
16/16 - 0s - loss: 0.4627 - accuracy: 0.7746 - 51ms/epoch - 3ms/step
Epoch 207/500
16/16 - 0s - loss: 0.4661 - accuracy: 0.7741 - 48ms/epoch - 3ms/step
Epoch 208/500
16/16 - 0s - loss: 0.4644 - accuracy: 0.7745 - 64ms/epoch - 4ms/step
Epoch 209/500
16/16 - 0s - loss: 0.4639 - accuracy: 0.7754 - 60ms/epoch - 4ms/step
Epoch 210/500
16/16 - 0s - loss: 0.4617 - accuracy: 0.7759 - 57ms/epoch - 4ms/step
Epoch 211/500
16/16 - 0s - loss: 0.4624 - accuracy: 0.7754 - 55ms/epoch - 3ms/step
Epoc

Epoch 299/500
16/16 - 0s - loss: 0.4531 - accuracy: 0.7806 - 43ms/epoch - 3ms/step
Epoch 300/500
16/16 - 0s - loss: 0.4536 - accuracy: 0.7818 - 44ms/epoch - 3ms/step
Epoch 301/500
16/16 - 0s - loss: 0.4532 - accuracy: 0.7810 - 47ms/epoch - 3ms/step
Epoch 302/500
16/16 - 0s - loss: 0.4519 - accuracy: 0.7801 - 40ms/epoch - 3ms/step
Epoch 303/500
16/16 - 0s - loss: 0.4506 - accuracy: 0.7814 - 50ms/epoch - 3ms/step
Epoch 304/500
16/16 - 0s - loss: 0.4517 - accuracy: 0.7830 - 54ms/epoch - 3ms/step
Epoch 305/500
16/16 - 0s - loss: 0.4533 - accuracy: 0.7818 - 36ms/epoch - 2ms/step
Epoch 306/500
16/16 - 0s - loss: 0.4506 - accuracy: 0.7824 - 47ms/epoch - 3ms/step
Epoch 307/500
16/16 - 0s - loss: 0.4512 - accuracy: 0.7809 - 47ms/epoch - 3ms/step
Epoch 308/500
16/16 - 0s - loss: 0.4513 - accuracy: 0.7816 - 45ms/epoch - 3ms/step
Epoch 309/500
16/16 - 0s - loss: 0.4509 - accuracy: 0.7821 - 47ms/epoch - 3ms/step
Epoch 310/500
16/16 - 0s - loss: 0.4515 - accuracy: 0.7813 - 44ms/epoch - 3ms/step
Epoc

Epoch 398/500
16/16 - 0s - loss: 0.4446 - accuracy: 0.7854 - 46ms/epoch - 3ms/step
Epoch 399/500
16/16 - 0s - loss: 0.4441 - accuracy: 0.7871 - 47ms/epoch - 3ms/step
Epoch 400/500
16/16 - 0s - loss: 0.4453 - accuracy: 0.7853 - 38ms/epoch - 2ms/step
Epoch 401/500
16/16 - 0s - loss: 0.4451 - accuracy: 0.7858 - 46ms/epoch - 3ms/step
Epoch 402/500
16/16 - 0s - loss: 0.4443 - accuracy: 0.7865 - 43ms/epoch - 3ms/step
Epoch 403/500
16/16 - 0s - loss: 0.4446 - accuracy: 0.7859 - 37ms/epoch - 2ms/step
Epoch 404/500
16/16 - 0s - loss: 0.4452 - accuracy: 0.7848 - 42ms/epoch - 3ms/step
Epoch 405/500
16/16 - 0s - loss: 0.4438 - accuracy: 0.7857 - 47ms/epoch - 3ms/step
Epoch 406/500
16/16 - 0s - loss: 0.4434 - accuracy: 0.7857 - 47ms/epoch - 3ms/step
Epoch 407/500
16/16 - 0s - loss: 0.4439 - accuracy: 0.7859 - 41ms/epoch - 3ms/step
Epoch 408/500
16/16 - 0s - loss: 0.4444 - accuracy: 0.7858 - 53ms/epoch - 3ms/step
Epoch 409/500
16/16 - 0s - loss: 0.4432 - accuracy: 0.7861 - 40ms/epoch - 2ms/step
Epoc

Epoch 497/500
16/16 - 0s - loss: 0.4394 - accuracy: 0.7889 - 43ms/epoch - 3ms/step
Epoch 498/500
16/16 - 0s - loss: 0.4394 - accuracy: 0.7883 - 38ms/epoch - 2ms/step
Epoch 499/500
16/16 - 0s - loss: 0.4394 - accuracy: 0.7901 - 39ms/epoch - 2ms/step
Epoch 500/500
16/16 - 0s - loss: 0.4387 - accuracy: 0.7899 - 41ms/epoch - 3ms/step


## Validation of the Model

In [13]:
#Compare Model performace between training and testing data
model_loss_train, model_accuracy_train = model.evaluate(X_train_scaled, y_train_categorical, verbose=2)
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"Train --> Loss: {model_loss_train}, Accuracy: {model_accuracy_train}")    
print(f"Test --> Loss: {model_loss}, Accuracy: {model_accuracy}")

964/964 - 1s - loss: 0.4374 - accuracy: 0.7899 - 747ms/epoch - 775us/step
322/322 - 0s - loss: 0.4901 - accuracy: 0.7699 - 234ms/epoch - 728us/step
Train --> Loss: 0.43737196922302246, Accuracy: 0.7898731827735901
Test --> Loss: 0.490070641040802, Accuracy: 0.7698744535446167


## Saving the Trained Model

In [14]:
# Save the model
model.save("../Models/h5/stempo&speech.h5")